# Default Rate Sensitivity on Marketing Spend Framework - Kenya

The effects of changing default rates on Liang & Zu's marketing spend framework is explored.

---

In [50]:
import pandas as pd
import numpy as np
import pyltv
from dbm import DBM
import plotly
from plotly import graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit

In [51]:
data = pd.read_csv('data/ke_data_liang.csv')

### Study Assumptions
1. Default rate stress applied as a blanket stress across all time periods.
2. Default rate stress applied to both 7dpd and 365dpd default rates.
3. The default rate stress is simply a percentage point added to the actual default rate. **So if the 365dpd default rate is currently 6%, adding a 1% stress results in a new 365dpd default rate of 7%**.

## Impact of Default Rate on LTV

In [52]:
min_months=4
n_months=50
default_stress=0.01

m1 = pyltv.Model(data, market='ke', fcast_method='powerslope')
m1.generate_features()
m1.forecast = m1.forecast_data(m1.data, min_months, n_months)

m2 = pyltv.Model(data, market='ke', fcast_method='powerslope', default_stress=default_stress)
m2.generate_features()
m2.forecast = m2.forecast_data(m2.data, min_months, n_months)

Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...


In [53]:
colors = {i: c for i, c in enumerate(plotly.colors.qualitative.Dark24)}

fig = make_subplots(rows=1, cols=2, shared_xaxes=True,
                   subplot_titles=('Unstressed', '+1% Default Rates'))

param = 'cumulative_dcf_ltv_per_original'

# LEFT PLOT
curves1=[]
for cohort in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort == cohort]
    for dtype in c_data.data_type.unique():
        output = c_data[c_data.data_type == dtype][param]

        output.name = cohort + '-' + dtype

        curves1.append(output)
    
for i, cohort in enumerate(curves1):
    c = colors[i]
    if 'forecast' in cohort.name:
        fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                 line=dict(width=3, dash='dash', color=c), legendgroup=f'{i}'), row=1, col=1)
    else:
        if cohort.notnull().any():
            fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                     line=dict(width=2, color=c), legendgroup=f'{i}'), row=1, col=1)

expected = m1.ltv_expected.cumulative_ltv_per_original
fig.add_trace(go.Scatter(name='expected', x=expected.index, y=expected, 
                         line=dict(color='black', width=3), legendgroup='e'), row=1, col=1)
fig.add_trace(go.Scatter(name='expected', x=expected.index, y=expected, 
                        line=dict(color='black', width=3), legendgroup='e', showlegend=False), row=1, col=2)

# RIGHT PLOT
curves2=[]
for cohort in m2.forecast.cohort.unique():
    c_data = m2.forecast[m2.forecast.cohort == cohort]
    for dtype in c_data.data_type.unique():
        output = c_data[c_data.data_type == dtype][param]

        output.name = cohort + '-' + dtype

        curves2.append(output)
        
for i, cohort in enumerate(curves2):
    c = colors[i]
    if 'forecast' in cohort.name:
        fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='lines',
                                 line=dict(width=3, dash='dash', color=c), legendgroup=f'{i}', showlegend=False), row=1, col=2,)
    else:
        if cohort.notnull().any():
            fig.add_trace(go.Scatter(name=cohort.name, x=cohort.index, y=cohort, mode='markers+lines',
                                     line=dict(width=2, color=c), legendgroup=f'{i}', showlegend=False), row=1, col=2)

fig.update_layout(title='DCF LTV')
fig.layout.xaxis.title='Month'
fig.layout.xaxis2.title='Month'
fig.update_yaxes(range=[-10, 55])

fig.show()

In [54]:
m1.plot_cohorts('default_rate_365dpd', data='forecast')
m2.plot_cohorts('default_rate_365dpd', data='forecast')

### Standard Default Rate

In [55]:
data_50mo1 = m1.forecast[m1.forecast['Months Since First Loan Disbursed']==50]
ltv1 = data_50mo1['cumulative_ltv_per_original'].median()
dcf_ltv1 = data_50mo1['cumulative_dcf_ltv_per_original'].median()
#print(f'50mo LTV: ${round(ltv1, 2)}')
print(f'Median DCF 50mo LTV: ${round(dcf_ltv1, 2)}')

Median DCF 50mo LTV: $35.84


### +1% Default Rate Stress

In [56]:
data_50mo2 = m2.forecast[m2.forecast['Months Since First Loan Disbursed']==50]
ltv2 = data_50mo2['cumulative_ltv_per_original'].median()
dcf_ltv2 = data_50mo2['cumulative_dcf_ltv_per_original'].median()
#print(f'50mo LTV: ${round(ltv2, 2)}')
print(f'Median DCF 50mo LTV: ${round(dcf_ltv2, 2)}')

Median DCF 50mo LTV: $27.53


In [57]:
print('Change in DCF LTV:')
print(f'{round(100*(dcf_ltv2-dcf_ltv1)/dcf_ltv1, 2)}%')

Change in DCF LTV:
-23.18%


In [58]:
ltvs = {}
for x in np.linspace(-0.03, 0.03, 7):
    m = pyltv.Model(data, market='ke', fcast_method='powerslope', default_stress=round(x, 2))

    m.generate_features()

    m.forecast = m.forecast_data(m.data, n_months=50)
    
    data_50mo = m.forecast[m.forecast['Months Since First Loan Disbursed']==50]
    ltvs[round(x, 2)] = (round(data_50mo['cumulative_dcf_ltv_per_original'].mean(), 2), 
                         round(100*data_50mo.default_rate_365dpd.mean(), 2))


Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...
Data spans 2020-09 to 2021-12
Total # of cohorts: 16
...


### Default Rate Stress: -3% to +3%

The plot below shows the relationship between a change in the default rate stress and the median DCF LTV for all cohorts.

In [59]:
v = [i[0] for i in ltvs.values()]
l = [f'{i[1]}%' for i in ltvs.values()]
fig = go.Figure(go.Scatter(x=l, y=v))
fig.update_layout(xaxis=dict(title='Default Rate Stress'),
                 yaxis=dict(title='Median DCF LTV'))
fig.show()

In [60]:
print('For every +1% increase in default rate...')
print(f'-${round(ltvs[0][0]-ltvs[0.01][0], 2)}')

For every +1% increase in default rate...
-$8.31


The relationships between LTV and default rates is shown below and explains the linear trend. **Note that this analysis does not include the effects of default rate changes on retention.**

LTV ~ [revenue - (origination+revenue)*default_rate_365dpd]

revenue ~ [0.08 * default_rate_7dpd]

## Default Rate Historicals

#### Weighted Average Lifetime Default

This method uses the average lifetime default rate (365dpd) for each cohort. The average lifetime default rate is a weighted average of defaults by origination_per_original.

In [61]:
defaults = []
default_changes = []
avg_lifetime_default = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c][['origination_per_original', 'default_rate_365dpd']]
    c_data['defaulted_origination'] = c_data['default_rate_365dpd']*c_data.origination_per_original
    c_data['defaulted_origination_change'] = (c_data.defaulted_origination-c_data.defaulted_origination.shift(1))/ \
        c_data.defaulted_origination
    
    defaults.append(go.Scatter(name=c, x=c_data.index, y=c_data.defaulted_origination))
    default_changes.append(go.Scatter(name=c, x=c_data.index, y=c_data.defaulted_origination_change))
    
    avg_lifetime_default.append(c_data.defaulted_origination.sum()/c_data.origination_per_original.sum())
    
avg_lifetime_default = pd.Series(avg_lifetime_default, index=m1.forecast.cohort.unique())

In [62]:
fig=go.Figure(defaults)
fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Defaulted Origination per Original'))
fig.show()

In [63]:
def reg(x, m, b):
    return m*x + b

params, covs = curve_fit(reg, xdata=np.arange(1, len(avg_lifetime_default)+1), ydata=avg_lifetime_default)

In [64]:
cohorts = []

for i, c in enumerate(avg_lifetime_default.index):
    cohorts.append(go.Scatter(name=c, x=[i+1], y=[avg_lifetime_default.iloc[i]], mode='markers'))

fig = go.Figure(cohorts + [go.Scatter(x=np.arange(1,36), y=reg(np.arange(1,36), params[0], params[1]))])

fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Default Rate'))
fig.show()

**1 year from now, at 24 months, Lifetime Default will be at 10.3%**. This can be used to buffer the LTV/mCAC ratio.

#### Unweighted Average Default

In this method, we simply look at aggregate stats of the 365 dpd default rate, without taking into account origination. This scenario looks at a blanket default rate increase across the board, regardless of the size of loans.

In [65]:
default_stats = {}
for i, c in enumerate(m1.forecast.cohort.unique()):
    c_data = m1.forecast[m1.forecast.cohort==c][['default_rate_365dpd']]
    
    default_stats[c] = (float(c_data.median()), float(c_data.quantile(0.9)))

In [66]:
medians=[]
quantile90=[]
for c in default_stats:
    medians.append(default_stats[c][0])
    quantile90.append(default_stats[c][1])
    
medians = pd.Series(medians, index=np.arange(1, len(medians)+1))
params1, covs1 = curve_fit(reg, xdata=medians.index, ydata=medians)

quantile90 = pd.Series(quantile90, index=np.arange(1, len(medians)+1))
params2, covs2 = curve_fit(reg, xdata=quantile90.index, ydata=quantile90)
    
fig = go.Figure([
    go.Scatter(name='median default', x=medians.index, y=medians, mode='markers'),
    go.Scatter(name='median-fit', x=np.arange(1,36), y=reg(np.arange(1,36), params1[0], params1[1])),
    go.Scatter(name='90 percentile', x=quantile90.index, y=quantile90, mode='markers'),
    go.Scatter(name='90 percentile-fit', x=np.arange(1,36), y=reg(np.arange(1,36), params2[0], params2[1]))
])

fig.update_layout(xaxis=dict(title='Month'), yaxis=dict(title='Default Rate'))
fig.show()

Similar to the previous method, we can project the median & 90% default rate levels to later months. **At 24 months, the 90 percentile default rate is 11.9%**

## Payback Period

### mCAC Payback Period

In [67]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 6.16
        elif r == 1.5:
            cac_ratio = 4.79
        elif r == 1:
            cac_ratio = 3.28
            
        avg_cac = float(dcf_ltv/cac_ratio)

        dcf = c_data['cumulative_dcf_ltv_per_original']
        payback_idx = dcf[dcf >= mcac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results1 = pd.DataFrame.from_records(paybacks)

In [68]:
ratios = [1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    c_data_stressed = m2.forecast[m2.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    dcf_ltv_stressed = float(c_data_stressed[c_data_stressed['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv, 'DCF LTV Stressed': dcf_ltv_stressed}
    for r in ratios:
        mcac = float(dcf_ltv/r)

        if r == 2:
            cac_ratio = 6.16
        elif r == 1.5:
            cac_ratio = 4.79
        elif r == 1:
            cac_ratio = 3.28
            
        avg_cac = float(dcf_ltv/cac_ratio)
        
        dcf = c_data_stressed['cumulative_dcf_ltv_per_original']
        
        payback_idx = dcf[dcf >= mcac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results2 = pd.DataFrame.from_records(paybacks)

The table below shows the payback periods for unstressed default rates in the 1.5x scenario.

In [69]:
ratio = 1.5
payback_results1[['cohort', 'DCF LTV', f'mCAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,mCAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,51.928789,34.619193,24,34.706665
1,2020-10,43.922846,29.281897,24,29.424495
2,2020-11,39.624650,26.416433,24,26.809791
3,2020-12,32.005983,21.337322,25,21.975402
4,2021-01,38.961313,25.974208,24,26.096746
5,2021-02,33.695659,22.463773,25,22.984885
6,2021-03,31.259747,20.839831,26,21.368169
7,2021-04,35.838122,23.892081,25,23.935083
8,2021-05,35.870470,23.913647,26,24.733676
9,2021-06,33.032262,22.021508,26,22.730812


The second table shows the new payback periods in the same 1.5x scenario, but with a +1% default rate stress applied. The payback periods are significantly pushed out. In the worst case (latest cohort, 2021-07), the payback period is pushed from 26 to 42 months!

In [70]:
ratio = 1.5
payback_results2[['cohort', 'DCF LTV', 'DCF LTV Stressed', f'mCAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,DCF LTV Stressed,mCAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,51.928789,42.749088,34.619193,32,34.891780
1,2020-10,43.922846,35.324610,29.281897,33,29.373667
2,2020-11,39.624650,30.985725,26.416433,35,26.744377
3,2020-12,32.005983,23.850435,21.337322,39,21.591201
4,2021-01,38.961313,30.455386,25.974208,35,26.171218
5,2021-02,33.695659,25.569216,22.463773,37,22.468354
6,2021-03,31.259747,23.301492,20.839831,39,20.840868
7,2021-04,35.838122,27.529807,23.892081,37,24.039383
8,2021-05,35.870470,27.550129,23.913647,37,24.015883
9,2021-06,33.032262,24.951889,22.021508,38,22.053788


In [71]:
fig1 = go.Figure([go.Scatter(name='1x', x=payback_results1['Payback Period (1)'], 
                            y=payback_results1['Payback Amount (1)'], mode='markers',
                            marker=dict(color='red')),
                  go.Scatter(name='1.5x', x=payback_results1['Payback Period (1.5)'], 
                            y=payback_results1['Payback Amount (1.5)'], mode='markers',
                            marker=dict(color='blue')),
                 go.Scatter(name='2x', x=payback_results1['Payback Period (2)'], 
                            y=payback_results1['Payback Amount (2)'], mode='markers',
                            marker=dict(color='orange'))
                ])

fig1.update_layout(title='Unstressed Default Rate')
fig1.update_layout(xaxis=dict(title='Month', range=[18, 44]), yaxis=dict(title='DCF LTV'))

fig2 = go.Figure([go.Scatter(name='1.5x', x=payback_results2['Payback Period (1.5)'], 
                            y=payback_results2['Payback Amount (1.5)'], mode='markers',
                            marker=dict(color='blue')),
                 go.Scatter(name='2x', x=payback_results2['Payback Period (2)'], 
                            y=payback_results2['Payback Amount (2)'], mode='markers',
                            marker=dict(color='orange'))
                ])

fig2.update_layout(title='+10% Default Rate')
fig2.update_layout(xaxis=dict(title='Month', range=[18, 44]), yaxis=dict(title='DCF LTV'))

fig1.show()
fig2.show()

In [72]:
payback_results1.set_index('cohort', drop=True).iloc[-1]

DCF LTV                 27.247784
mCAC (1)                27.247784
CAC (1)                  8.307251
Payback Period (1)      50.000000
Payback Amount (1)      27.247784
mCAC (1.5)              18.165189
CAC (1.5)                5.688473
Payback Period (1.5)    26.000000
Payback Amount (1.5)    18.166996
mCAC (2)                13.623892
CAC (2)                  4.423342
Payback Period (2)      21.000000
Payback Amount (2)      14.061747
Name: 2021-07, dtype: float64

In [73]:
payback_results2.set_index('cohort', drop=True).iloc[-1]

DCF LTV                 27.247784
DCF LTV Stressed        19.726180
mCAC (1.5)              18.165189
CAC (1.5)                5.688473
Payback Period (1.5)    42.000000
Payback Amount (1.5)    18.236025
mCAC (2)                13.623892
CAC (2)                  4.423342
Payback Period (2)      29.000000
Payback Amount (2)      13.680077
Name: 2021-07, dtype: float64

In the worst case, our most recent cohort, the **payback period will shift out 26 to 42 months (16 months) at an LTV/mCAC of 1.5**, and from **21 to 29 (8 days) for an LTV/mCAC of 2**.

### Average CAC Payback Period

In [75]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv}
    for r in ratios:
        mcac = float(dcf_ltv/r)
        if r == 2:
            cac_ratio = 6.16
        elif r == 1.5:
            cac_ratio = 4.79
        elif r == 1:
            cac_ratio = 3.28
            
        avg_cac = float(dcf_ltv/cac_ratio)

        dcf = c_data['cumulative_dcf_ltv_per_original']
        payback_idx = dcf[dcf >= avg_cac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results1 = pd.DataFrame.from_records(paybacks)

In [76]:
ratios = [1, 1.5, 2]
paybacks = []
for c in m1.forecast.cohort.unique():
    c_data = m1.forecast[m1.forecast.cohort==c]
    c_data_stressed = m2.forecast[m2.forecast.cohort==c]
    
    dcf_ltv = float(c_data[c_data['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    dcf_ltv_stressed = float(c_data_stressed[c_data_stressed['Months Since First Loan Disbursed']==50]['cumulative_dcf_ltv_per_original'])
    
    record = {'cohort': c, 'DCF LTV': dcf_ltv, 'DCF LTV Stressed': dcf_ltv_stressed}
    for r in ratios:
        mcac = float(dcf_ltv/r)

        if r == 2:
            cac_ratio = 6.16
        elif r == 1.5:
            cac_ratio = 4.79
        elif r == 1:
            cac_ratio = 3.28
            
        avg_cac = float(dcf_ltv/cac_ratio)
        
        dcf = c_data_stressed['cumulative_dcf_ltv_per_original']
        
        payback_idx = dcf[dcf >= avg_cac].index[0]

        record[f'mCAC ({r})'] = mcac
        record[f'CAC ({r})'] = avg_cac
        record[f'Payback Period ({r})'] = payback_idx
        record[f'Payback Amount ({r})'] = dcf.loc[payback_idx]
    
    paybacks.append(record)
    
payback_results2 = pd.DataFrame.from_records(paybacks)

The two tables below show the changes in payback periods when using LTV/CAC instead of LTV/mCAC as the guard rail. The shifts in payback periods are much less significant.

In [77]:
ratio = 1.5
payback_results1[['cohort', 'DCF LTV', f'CAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,CAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,51.928789,10.841083,10,11.039632
1,2020-10,43.922846,9.169696,11,10.460121
2,2020-11,39.624650,8.272370,11,9.280554
3,2020-12,32.005983,6.681834,12,7.976103
4,2021-01,38.961313,8.133886,11,8.493802
5,2021-02,33.695659,7.034584,12,7.947541
6,2021-03,31.259747,6.526043,13,7.129253
7,2021-04,35.838122,7.481863,13,8.843860
8,2021-05,35.870470,7.488616,13,8.596582
9,2021-06,33.032262,6.896088,13,7.861995


In [78]:
ratio = 1.5
payback_results2[['cohort', 'DCF LTV', 'DCF LTV Stressed', f'CAC ({ratio})', f'Payback Period ({ratio})', f'Payback Amount ({ratio})']]

,cohort,DCF LTV,DCF LTV Stressed,CAC (1.5),Payback Period (1.5),Payback Amount (1.5)
0,2020-09,51.928789,42.749088,10.841083,13,10.868693
1,2020-10,43.922846,35.324610,9.169696,14,10.207653
2,2020-11,39.624650,30.985725,8.272370,14,9.338812
3,2020-12,32.005983,23.850435,6.681834,15,6.933781
4,2021-01,38.961313,30.455386,8.133886,14,8.604194
5,2021-02,33.695659,25.569216,7.034584,15,7.163777
6,2021-03,31.259747,23.301492,6.526043,17,7.124878
7,2021-04,35.838122,27.529807,7.481863,16,8.091425
8,2021-05,35.870470,27.550129,7.488616,16,7.893343
9,2021-06,33.032262,24.951889,6.896088,16,6.944364


In [79]:
fig1 = go.Figure([go.Scatter(name='1x', x=payback_results1['Payback Period (1)'], 
                            y=payback_results1['Payback Amount (1)'], mode='markers'),
                  go.Scatter(name='1.5x', x=payback_results1['Payback Period (1.5)'], 
                            y=payback_results1['Payback Amount (1.5)'], mode='markers'),
                 go.Scatter(name='2x', x=payback_results1['Payback Period (2)'], 
                            y=payback_results1['Payback Amount (2)'], mode='markers')
                ])

fig1.update_layout(title='Unstressed Default Rate')
fig1.update_layout(xaxis=dict(title='Month', range=[8, 22]), yaxis=dict(title='DCF LTV'))

fig2 = go.Figure([go.Scatter(name='1x', x=payback_results2['Payback Period (1)'], 
                            y=payback_results2['Payback Amount (1)'], mode='markers'),
                  go.Scatter(name='1.5x', x=payback_results2['Payback Period (1.5)'], 
                            y=payback_results2['Payback Amount (1.5)'], mode='markers'),
                 go.Scatter(name='2x', x=payback_results2['Payback Period (2)'], 
                            y=payback_results2['Payback Amount (2)'], mode='markers')
                ])

fig2.update_layout(title='+10% Default Rate')
fig2.update_layout(xaxis=dict(title='Month', range=[8, 22]), yaxis=dict(title='DCF LTV'))

fig1.show()
fig2.show()

In [80]:
payback_results1.set_index('cohort', drop=True).iloc[-1]

DCF LTV                 27.247784
mCAC (1)                27.247784
CAC (1)                  8.307251
Payback Period (1)      16.000000
Payback Amount (1)       8.652526
mCAC (1.5)              18.165189
CAC (1.5)                5.688473
Payback Period (1.5)    14.000000
Payback Amount (1.5)     6.306132
mCAC (2)                13.623892
CAC (2)                  4.423342
Payback Period (2)      13.000000
Payback Amount (2)       5.153027
Name: 2021-07, dtype: float64

In [81]:
payback_results2.set_index('cohort', drop=True).iloc[-1]

DCF LTV                 27.247784
DCF LTV Stressed        19.726180
mCAC (1)                27.247784
CAC (1)                  8.307251
Payback Period (1)      21.000000
Payback Amount (1)       8.564643
mCAC (1.5)              18.165189
CAC (1.5)                5.688473
Payback Period (1.5)    18.000000
Payback Amount (1.5)     5.926285
mCAC (2)                13.623892
CAC (2)                  4.423342
Payback Period (2)      17.000000
Payback Amount (2)       4.955956
Name: 2021-07, dtype: float64

In the worst case, our most recent cohort, the **payback period will shift out 14 to 18 months (4 months) at an LTV/CAC of 1.5**, and from **13 to 17 months (4 months) for an LTV/CAC of 2**.

## Conclusions

- A rise in default rate leads to an increas in payback period. This is true for any scenario (1x, 1.5x, 2x). In general, the shift in payback period is worse for lower multipliers (less margin).
- A +1% increase in default rate in the 1.5x scenario will shift the payback period from 26 to 42 months (+16 months). On average (across cohorts), we see a shift in payback periods of +12 months.
- With a 1.5x scenario, the data suggests we can still maintain a 50mo positive LTV inspite of up to a +1% rise in default rates.